# BERT Text Classification: Classify spam vs no spam emails

In [ ]:
# @title
import tensorflow as tf
from tensorflow import keras
import numpy as np
print('New Notebook file and type code, the GPU Mode is reset...')
print('devices:', tf.config.experimental.list_physical_devices())

In [ ]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# Import the dataset (Dataset is taken from kaggle)¶

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/MyDrive')
# First, put the span.csv in the google drive:
# drive.google.com/drive/my-drive/My Drive
# Verify the path name inside colab: MyDrive/MyDrive/spam.csv
df = pd.read_csv("/content/MyDrive/MyDrive/spam.csv")
df.head(5)

In [ ]:
df.groupby('Category').describe()

In [ ]:
print("df['Category'].shape:")
print(df['Category'].shape)
print()
print("df.sample(5):")
print(df.sample(5))
print()
print("df['Category'].value_counts():")
print(df['Category'].value_counts())

In [ ]:
print('747/4835):', 747/4835)
print('4835/(747+4835):', 4835/(747+4835))
print('747/(747+4835):', 747/(747+4835))
print('4835/(747+4835):', 4835/(747+4835))

In [ ]:
df_spam = df[df['Category']=='spam']
print('df_spam.shape:', df_spam.shape)

In [ ]:
df_ham = df[df['Category']=='ham']
print('df_ham.shape:', df_ham.shape)

In [ ]:
# we can take 2 sample from df_ham
df_ham.sample(2)

In [ ]:
# We take dowmsample pf ham
df_spam = df[df['Category']=='spam']
print('df_spam.shape:', df_spam.shape)
df_ham_downsampled = df_ham.sample(df_spam.shape[0])

print('df_spam.shape[0]:', df_spam.shape[0])
print('df_ham_downsampled.shape:', df_ham_downsampled.shape)

In [ ]:
df_balanced = pd.concat([df_spam, df_ham_downsampled])
print('df_balanced.shape:', df_balanced.shape)
print()
print("df_balanced['Category'].value_counts():")
print(df_balanced['Category'].value_counts())

In [ ]:
df_balanced.sample(5)

In [ ]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x == 'spam' else 0)
print('df_balanced.sample(5):')
print(df_balanced.sample(5))

In [ ]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

# Split it into training and test data set

In [ ]:
#rom sklearn.model_selection import train_test_split
#
#X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df['spam'])
#
#X_train.head(4)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

X_train.head(4)

# Now lets import BERT model and get embeding vectors for few sample statements¶

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

# Get embeding vectors for few sample words. Compare them using cosine similarity

In [ ]:
e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)
print('banana: len(e[0]):     ', len(e[0]))
print('banana: e[0].shape:    ', e[0].shape)
print('grapes: e[1].shape:    ', e[1].shape)
print('mango: e[2].shape:     ', e[2].shape)
print('jeff bezos: e[3].shape:', e[3].shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
print('*****************Fruits Similarity************************************')
print('Banana vs grapes:')
print('cosine_similarity([e[0]],[e[1]]:', cosine_similarity([e[0]],[e[1]]))  # Banana vs grapes
print('Banana vs Mongo:')
print('cosine_similarity([e[0]],[e[2]]:', cosine_similarity([e[0]],[e[2]]))  # Banana vs Mongo
print()
print('*****************Entrepreneur Similairty******************************')
print('Jeff Bezos vs Elon Musk:')
print('cosine_similarity([e[3]],[e[4]]:', cosine_similarity([e[3]],[e[4]]))  # Jeff Bezos vs Elon Musk
print('Jeff Bezos vs Bill Gates:')
print('cosine_similarity([e[3]],[e[5]]:', cosine_similarity([e[3]],[e[5]]))  # Jeff Bezos vs Bill Gates
print()
print('******************Fruite vs Entrepreneur Similairty*******************')
print('Banana vs Elon Musk:')
print('cosine_similarity([e[0]],[e[4]]:', cosine_similarity([e[0]],[e[4]]))  # Banana vs Elon Musk
print('Grapes vs Elon Musk:')
print('cosine_similarity([e[1]],[e[4]]:', cosine_similarity([e[1]],[e[4]]))  # Grapes vs Elon Musk

# Values near to 1 means they are similar. 0 means they are very different. Above you can use comparing "banana" vs "grapes" you get 0.99 similarity as they both are fruits

In [ ]:
print('******************Fruite vs Entrepreneur Similairty*******************')
print('Banana vs Jeff Bezos:')
cosine_similarity([e[0]],[e[3]])

# Comparing banana with jeff bezos you still get 0.84 but it is not as close as 0.99 that we got with grapes

In [ ]:
cosine_similarity([e[3]],[e[4]])

# Jeff bezos and Elon musk are more similar then Jeff bezos and banana as indicated above

# Build Model
There are two types of models you can build in tensorflow.

(1) Sequential (2) Functional

So far we have built sequential model. But below we will build functional model. More information on these two is here: https://becominghuman.ai/sequential-vs-functional-model-in-keras-20684f766057

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

# https://stackoverflow.com/questions/47605558/importerror-failed-to-import-pydot-you-must-install-pydot-and-graphviz-for-py

In [ ]:
model.summary()

In [ ]:
METRICS = [
   tf.keras.metrics.BinaryAccuracy(name='accuracy'),
   tf.keras.metrics.Precision(name='precision'),
   tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
#len(X_train)

In [ ]:
#model.compile(optimizer='adam',
#              loss='binary_crossentropy',
#              metrics=['accuracy'])

# Train the model

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
# Perform the Model Prediction for X_test
print('X_test[:5]:')
print(X_test[:5])
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print('y_predicted[:5]:')
print(y_predicted[:5])

In [ ]:
# COnvert the predict probabilty into (1, 0)
import numpy as np
y_predicted = np.where(y_predicted > 0.5, 1, 0)
print('y_predicted into (1, 0) [:5]:')
print(y_predicted[:5])


In [ ]:
# Print the COnfusion Matrix
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_predicted)
print('cm:')
print(cm)

In [ ]:
# Draw the confusiosn Matrix
from matplotlib import pyplot as plt
import seaborn as sn
# fmt: format, 'd' = display in digit (integer)
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel("Predicted")
plt.ylabel("Truth")
plt.show()

# Inference

Note: Â£: Pound sign.

In [ ]:
reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)